In [ ]:
import json
import numpy as np

with open('../config.json') as f:
    config = json.load(f)

In [ ]:
config['env']['observation_map']

In [ ]:
from neurosim.game_interface import _parse_rf_map

obs_rf = [_parse_rf_map(func_def) for func_def in config['env']['observation_map']]

In [ ]:
def find_boundary(idx, f, imin=None, imax=None, eps=1e-6):
    if imin == None:
        imin = -10000
    if imax == None:
        imax = 10000
        
    midpoint = imin + (imax - imin) / 2
    if f(imin).index(1.0) < idx:
        while f(midpoint).index(1.0) > idx:
            midpoint = imin + (midpoint - imin) / 2
        return find_boundary(idx, f, midpoint, imax, eps)
    if f(imax).index(1.0) > idx + 1:
        while f(midpoint).index(1.0) < idx + 1:
            midpoint = midpoint + (imax - midpoint) / 2
        return find_boundary(idx, f, imin, midpoint, eps)
    
    if imax - imin < eps:
        return round(imin, 5), imin, imax
    
    if f(midpoint).index(1.0) == idx:
        return find_boundary(idx, f, midpoint, imax, eps)
    if f(midpoint).index(1.0) == idx + 1:
        return find_boundary(idx, f, imin, midpoint, eps)
    

In [ ]:
find_boundary(5, obs_rf[0])

In [ ]:
space = []
for obs_idx,obs in enumerate(config['env']['observation_map']):
    obs_space = []
    for idx in range(obs['bins'] - 1):
        boundary, _, _ = find_boundary(idx, obs_rf[obs_idx])
        obs_space.append(boundary)
    space.append(obs_space)

In [ ]:
sp = np.array(space)

import pickle as pkl

with open('obs_space.pkl', 'wb') as out:
    pkl.dump(sp, out)

In [ ]:
sp[:, 9:12]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

obs_names = ['Position', 'Velocity', 'Angle', 'Angular Velocity']
colors = ['b', 'g', 'r', 'y']

_,axs = plt.subplots(nrows=2, ncols=2, figsize=(12,10))

for idx1,axr in enumerate(axs):
    for idx2,ax in enumerate(axr):
        idx = idx1*len(axr) + idx2
        intervals = []
        d = space[idx][1] - space[idx][0]
        intervals.append([space[idx][0] - 1.75 * d, space[idx][0]])
        for si in range(len(space[idx])-1):
            intervals.append([space[idx][si], space[idx][si+1]])
        si = len(space[idx])-1
        d = space[idx][si] - space[idx][si-1]
        intervals.append([space[idx][si], space[idx][si] + 1.75 * d])


        x = np.arange(1, len(intervals)+1)
        height = [i2-i1 for i1, i2 in intervals]
        bottom = [i1 for i1, i2 in intervals]

        ax.bar(x,height, bottom=bottom, color=colors[idx])
        ax.set_ylim([min(intervals)[0], max(intervals)[1]])
        ax.set_xticks(x)
        ax.set_xlabel('Neuron ID')
        ax.set_ylabel('Receptive Field')
        ax.legend([obs_names[idx]])

        ax.grid(alpha=0.4)
        
plt.suptitle('Receptive Fields of Neurons ES population')
plt.tight_layout()
plt.show()

In [ ]:
obs_rf[0](-0.47654547839024697).index(1.0)

In [ ]:
obs_rf[0](-0.3239503162583135).index(1.0)

In [ ]:
import os
import csv
import numpy as np

wdirs = '../results/hpsearch-2021-09-01/'
config_file = '../config.json'

run_dirs = [os.path.join(wdirs, d) for d in os.listdir(wdirs) if d.startswith('run_')]

In [ ]:
from neurosim.game_interface import GameInterface
import gym
import json

class MockAIGame:
    observations = []
    rewards = []
    env = None
    def __init__(self, env):
        self.env = env

with open(config_file) as f:
    config = json.load(f)

env = gym.make('CartPole-v1')
game = MockAIGame(env)
gi = GameInterface(game, config)

In [ ]:
import math
BASE = 20 # The number of neurons
OBS_CNT = 4
CNST_SUM = np.array([i*BASE for i in range(OBS_CNT)])
CNST_PROD = np.array([BASE**i for i in range(OBS_CNT)])

def obsfrind_to_obscode(obsfrind):
    assert sum([o >= BASE for o in obsfrind]) == 0
    return np.sum(obsfrind * CNST_PROD)

def obscode_to_obsfrind(obscode):
    arr = []
    curr = obscode
    for i in range(OBS_CNT):
        arr.append(curr % BASE)
        curr = math.floor(curr / BASE)
    return np.array(arr)


obs_test = [
    np.array([10, 13, 8, 7]),
    np.array([19, 19, 19, 19]),
    np.array([0, 0, 0, 0]),
    np.array([0, 1, 2, 3])]
for o in obs_test:
    assert np.sum(obscode_to_obsfrind(obsfrind_to_obscode(o)) == o) == 4
    
MAX_OBS = obsfrind_to_obscode(np.array([19, 19, 19, 19])) + 1
print('MAX_OBS', MAX_OBS)

In [ ]:
SPACE = {}

In [ ]:
run_dirs = ['../results/hpsearch-2021-09-06/best/1_run_168/evaluation_8/']

In [ ]:
import json

for run_dir in run_dirs:
    # Read
    actionsPerEps = []
    with open(os.path.join(run_dir, 'ActionsPerEpisode.txt')) as f:
        for row in csv.reader(f, delimiter='\t'):
            actionsPerEps.append(int(float(row[1])))
    print(run_dir, 'actionsPerEps', len(actionsPerEps))

    actions = []
    with open(os.path.join(run_dir, 'ActionsRewards.txt')) as f:
        for row in csv.reader(f, delimiter='\t'):
            move = int(float(row[1]))
            obs = json.loads(row[-1])
            actions.append((move, obs))
    print(run_dir, 'actions', len(actions))

    # Process

    curr_step = 0
    for eps in actionsPerEps:
        first_step = 1 if curr_step == 0 else 0
        acts = actions[curr_step:curr_step+eps-first_step]
        codes = []
        for _,obs in acts:
            game.observations = [obs]
            fr = gi.input_firing_rates()
            frind = (fr > 0).nonzero()[0] - CNST_SUM
            codes.append(obsfrind_to_obscode(frind))

        for idx,((move,_), code) in enumerate(zip(acts,codes)):
            next_code = codes[idx+1] if idx + 1 < len(codes) else MAX_OBS
            if code not in SPACE:
                SPACE[code] = {}
            if move not in SPACE[code]:
                SPACE[code][move] = {}
            if next_code not in SPACE[code][move]:
                SPACE[code][move][next_code] = 0
            SPACE[code][move][next_code] += 1
        curr_step += eps-first_step
        
    print('SPACE size', len(SPACE))

In [ ]:
cnt_two_moves = 0
cnt_one_move = 0
for k,v in SPACE.items():
    if len(v) > 1:
        cnt_two_moves += 1
    else:
        cnt_one_move += 1
        
print('cnt_one_move', cnt_one_move)
print('cnt_two_moves', cnt_two_moves)

In [ ]:
def investigate(code, _space=SPACE):
    print(code, obscode_to_obsfrind(code))
    for move, outputs in sorted(list(_space[code].items()), key=lambda x:x[0]):
        tot = sum(outputs.values())
        print('move', move)
        for next_code, cnt in sorted(list(outputs.items()), key=lambda x:x[1], reverse=True):
            print('  {}, {}: {:.4f}'.format(next_code,  obscode_to_obsfrind(next_code), cnt / tot))
        
        
investigate(0)

In [ ]:
obsfrind_to_obscode(obscode_to_obsfrind(20))

In [ ]:
from tqdm import tqdm

SPACE_X = {}
SPACE_ANG = {}

def decode(space_map, code_map, obs, i1, i2):
    pre_code = obsfrind_to_obscode(np.array(list(obs[i1:i2]) + [0, 0]))
    if pre_code not in space_map:
        space_map[pre_code] = {}
    for move, res_map in code_map.items():
        if move not in space_map[pre_code]:
            space_map[pre_code][move] = {}
        for next_code, cnt in res_map.items():
            next_obs = obscode_to_obsfrind(next_code)
            post_code = obsfrind_to_obscode(np.array(list(next_obs[i1:i2]) + [0, 0]))
            if post_code not in space_map[pre_code][move]:
                space_map[pre_code][move][post_code] = 0
            space_map[pre_code][move][post_code] += cnt

for code, code_map in tqdm(SPACE.items()):
    obs = obscode_to_obsfrind(code)
    decode(SPACE_X, code_map, obs, 0, 2)
    decode(SPACE_ANG, code_map, obs, 2, 4)

In [ ]:
SPACE_X

In [ ]:
investigate(210, SPACE_X)

In [ ]:
investigate(210, SPACE_ANG)

In [ ]:
investigate(210 + 175 * 400)